In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/END

/content/drive/MyDrive/END


In [3]:
%%bash

python -m spacy download en
python -m spacy download de

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')


In [4]:
import torch
import numpy as np
import pandas as pd
import spacy
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchtext
from torchtext.data import Field, BucketIterator
import random
import math
import time

from torchtext.datasets import Multi30k

In [5]:
SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [6]:
spacy_en = spacy.load('en')
spacy_de = spacy.load('de')

In [7]:
def tokenize_en(text):
  return [token.text for token in spacy_en.tokenizer(text)]

def tokenize_de(text):
  return [token.text for token in spacy_de.tokenizer(text)]

In [8]:
SRC = Field(tokenize = tokenize_en, init_token='<sos>', eos_token='<eos>', lower = True)
TRG = Field(tokenize = tokenize_de, init_token='<sos>', eos_token='<eos>', lower = True)

In [9]:
train_data, valid_data, test_data = Multi30k.splits(root = '/content/drive/MyDrive/END/data/multi30k', exts = ('.en', '.de'), fields = (SRC, TRG))

In [10]:
vars(train_data.examples[0])

{'src': ['two',
  'young',
  ',',
  'white',
  'males',
  'are',
  'outside',
  'near',
  'many',
  'bushes',
  '.'],
 'trg': ['zwei',
  'junge',
  'weiße',
  'männer',
  'sind',
  'im',
  'freien',
  'in',
  'der',
  'nähe',
  'vieler',
  'büsche',
  '.']}

In [11]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 

In [13]:
batch_size = 128
train_iterator, valid_iterator, test_iterator = BucketIterator.splits((train_data, valid_data, test_data), 
                                                                      batch_size=batch_size, 
                                                                      device = device, 
                                                                      sort_within_batch=False)

In [14]:
len(SRC.vocab), len(TRG.vocab)

(5893, 7855)

In [15]:
len(train_data), len(valid_data), len(test_data)

(29000, 1014, 1000)

In [16]:
print(f"Source Sentence >>>>>{train_data.examples[0].src}")
print(f"Target Sentence >>>>>{train_data.examples[0].trg}")

Source Sentence >>>>>['two', 'young', ',', 'white', 'males', 'are', 'outside', 'near', 'many', 'bushes', '.']
Target Sentence >>>>>['zwei', 'junge', 'weiße', 'männer', 'sind', 'im', 'freien', 'in', 'der', 'nähe', 'vieler', 'büsche', '.']


In [17]:
one_batch = next(iter(train_iterator))

In [18]:
one_batch.src.shape, one_batch.trg.shape

(torch.Size([28, 128]), torch.Size([33, 128]))

## Building the Seq2Seq Model

### Encoder
![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq8.png?raw=1)



In [19]:
class Encoder(nn.Module):
  def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
    super().__init__()

    self.embedding = nn.Embedding(input_dim, emb_dim)
    self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional=True)
    self.fc = nn.Linear(enc_hid_dim*2, dec_hid_dim)
    self.dropout = nn.Dropout(dropout)
  def forward(self, src):
    # embedded = [src_len, batch_size, emb_dim]
    embedded = self.dropout(self.embedding(src))

    # outputs = [src_len, batch_size, enc_hid_dim * num_of_directions]
    # hidden = [num_of_layers * num_of_directions, batch_size, enc_hid_dim]
    outputs, hidden = self.rnn(embedded)

    # Concatenating the forward and backward hidden vectors and passing through linear layer to obtain
    # a combined context vector to pass to the decoder as its initial hidden state

    # hidden = [batch_size, dec_hid_dim]
    hidden = self.fc(torch.tanh(torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)))

    # return embedded, outputs, hidden
    return outputs, hidden

In [20]:
encoder_example = Encoder(input_dim=len(SRC.vocab), emb_dim = 128, enc_hid_dim = 256, dec_hid_dim=512, dropout = 0.5)
src_example = one_batch.src.cpu()
src_example.shape

torch.Size([28, 128])

In [21]:
embedding_example, output_ex, hidden_ex = encoder_example(src_example)
embedding_example.shape, output_ex.shape, hidden_ex.shape

(torch.Size([28, 128, 128]),
 torch.Size([28, 128, 512]),
 torch.Size([2, 128, 256]))

In [23]:
full_encoder = Encoder(input_dim=len(SRC.vocab), emb_dim = 128, enc_hid_dim = 256, dec_hid_dim=512, dropout = 0.5)
output_full, hidden_full = full_encoder(src_example)

### Attention

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq9.png?raw=1)

In [31]:
class Attention(nn.Module):
  def __init__(self, enc_hid_dim, dec_hid_dim):
    super().__init__()
    self.attn = nn.Linear(enc_hid_dim*2 + dec_hid_dim, dec_hid_dim)
    self.v = nn.Linear(dec_hid_dim, 1, bias = False)

  def forward(self, hidden, encoder_outputs):
    
    # encoder_outputs = [src_len, batch_size, enc_hid_dim*2]
    # hidden = [batch_size, dec_hid_dim]

    src_len = encoder_outputs.shape[0]
    
    hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
    encoder_outputs = encoder_outputs.permute(1, 0, 2)

    # encoder_outputs = [batch_size, src_len, enc_hid_dim*2]
    # hidden = [batch_size, src_len, dec_hid_dim]

    energy = self.attn(torch.tanh(torch.cat((encoder_outputs, hidden), dim = 2)))
    # energy = [batch_size, src_len, dec_hid_dim]

    attention = self.v(energy).squeeze(2)
    # attention = [batch_size, src_len]
    return F.softmax(attention, dim = 1)

In [35]:
output_full.shape, hidden_full.shape

(torch.Size([27, 128, 512]), torch.Size([128, 512]))

In [42]:
hidden_full.unsqueeze(1).repeat(1, output_full.shape[0], 1).shape

torch.Size([128, 27, 512])

In [54]:
attention_example = Attention(enc_hid_dim=256, dec_hid_dim=512)
attention_example

Attention(
  (attn): Linear(in_features=1024, out_features=512, bias=True)
  (v): Linear(in_features=512, out_features=1, bias=False)
)

In [49]:
energy = attention_example(hidden_full, output_full)
energy.shape

torch.Size([128, 27, 512])

In [52]:
v = nn.Linear(512, 1, bias = False)
v(energy).shape, v(energy).squeeze(2).shape

(torch.Size([128, 27, 1]), torch.Size([128, 27]))

In [56]:
F.softmax(v(energy).squeeze(2), dim = 1).shape

torch.Size([128, 27])

### Decoder

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq10.png?raw=1)


In [25]:
encoder_test = Encoder(len(SRC.vocab), emb_dim = 128, enc_hid_dim = 256, dec_hid_dim = 512, dropout = 0.1)
attention_test = Attention(enc_hid_dim=256, dec_hid_dim=512)
src_test = one_batch.src.cpu()

In [27]:
encoder_outputs, enc_hidden  = encoder_test(src_test)
attention = attention_test(enc_hidden, encoder_outputs)

print(f"Encoder Outputs Shape = {encoder_outputs.shape}")
print(f"Encoder Hidden Shape = {enc_hidden.shape}")
print(f"Attention Shape = {attention.shape}")

Encoder Outputs Shape = torch.Size([28, 128, 512])
Encoder Hidden Shape = torch.Size([128, 512])
Attention Shape = torch.Size([128, 28])


In [21]:
class Decoder(nn.Module):
  def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
    super().__init__()
    self.enc_hid_dim = enc_hid_dim
    self.output_dim = output_dim
    self.attention = attention
    self.embedding = nn.Embedding(output_dim, emb_dim)
    self.rnn = nn.GRU((enc_hid_dim*2) + emb_dim , dec_hid_dim)
    self.fc = nn.Linear((enc_hid_dim*2) + emb_dim + dec_hid_dim, output_dim)
    self.dropout = nn.Dropout(dropout)

  def forward(self, input, hidden, encoder_outputs):

    # input = [batch_size]
    # hidden = [batch_size, dec_hid_dim]
    # encoder_outputs = [src_len, batch_size, enc_hid_dim*2]

    a = self.attention(hidden, encoder_outputs)
    # a = [batch_size, src_len]

    a = a.unsqueeze(1)
    #a = [batch_size, 1, src_len]

    encoder_outputs = encoder_outputs.permute(1, 0, 2)
    # encoder_outputs = [batch_size, src_len, enc_hid_dim*2]

    weighted = torch.bmm(a, encoder_outputs)
    # weighted = [batch_size, 1, enc_hid_dim*2]

    input = input.unsqueeze(0)
    # input = [1, batch_size]
    embedded = self.dropout(self.embedding(input))
    # embedded = [1, batch_size, emb_dim]


    # To concatenate embedding and weighted, we need to permute the weighted
    weighted = weighted.permute(1, 0, 2)
    # weighted = [1, batch_size, enc_hid_dim*2]

    rnn_input = torch.cat((embedded, weighted), dim=2)
    # rnn_input = [1, batch_size, (enc_hidden_dim*2) + emb_dim]
    output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
    # outputs = [1, batch_size, dec_hid_dim]
    # hidden = [1, batch_size, dec_hid_dim]

    embedded = embedded.squeeze(0)
    output = output.squeeze(0)
    weighted = weighted.squeeze(0)
    
    prediction = self.fc(torch.cat((output, weighted, embedded), dim = 1))
    #prediction = [batch_size, output_dim]
    return prediction, hidden.squeeze(0)

In [57]:
decoder_test = Decoder(output_dim = len(TRG.vocab), 
                       emb_dim = 128, 
                       enc_hid_dim = 256, 
                       dec_hid_dim = 512, 
                       dropout = 0.1, 
                       attention = attention_test)

In [51]:
trg_test = one_batch.trg
trg_one_word = trg_test[1, :].cpu()
trg_one_word.shape

torch.Size([128])

In [44]:
decoder_outputs, decoder_hidden = decoder_test(trg_one_word, enc_hidden, encoder_outputs)
decoder_outputs.shape, decoder_hidden.shape

(torch.Size([1, 128, 512]), torch.Size([1, 128, 512]))

In [58]:
prediction, dec_hidden = decoder_test(trg_one_word, enc_hidden, encoder_outputs)
prediction.shape

torch.Size([128, 7855])

In [32]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
        
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src)
                
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and all encoder hidden states
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, encoder_outputs)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

In [33]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)

In [34]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(5893, 256)
    (rnn): GRU(256, 512, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(7855, 256)
    (rnn): GRU(1280, 512)
    (fc): Linear(in_features=1792, out_features=7855, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [35]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 24,036,783 trainable parameters


In [36]:
optimizer = optim.AdamW(model.parameters())
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [37]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [38]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [39]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [40]:
N_EPOCHS = 20
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'S10_modelA.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 1m 38s
	Train Loss: 5.064 | Train PPL: 158.203
	 Val. Loss: 4.906 |  Val. PPL: 135.120
Epoch: 02 | Time: 1m 39s
	Train Loss: 4.161 | Train PPL:  64.120
	 Val. Loss: 4.398 |  Val. PPL:  81.296
Epoch: 03 | Time: 1m 39s
	Train Loss: 3.666 | Train PPL:  39.107
	 Val. Loss: 4.115 |  Val. PPL:  61.233
Epoch: 04 | Time: 1m 39s
	Train Loss: 3.206 | Train PPL:  24.680
	 Val. Loss: 3.728 |  Val. PPL:  41.602
Epoch: 05 | Time: 1m 39s
	Train Loss: 2.841 | Train PPL:  17.141
	 Val. Loss: 3.586 |  Val. PPL:  36.105
Epoch: 06 | Time: 1m 39s
	Train Loss: 2.531 | Train PPL:  12.565
	 Val. Loss: 3.510 |  Val. PPL:  33.456
Epoch: 07 | Time: 1m 39s
	Train Loss: 2.234 | Train PPL:   9.333
	 Val. Loss: 3.411 |  Val. PPL:  30.289
Epoch: 08 | Time: 1m 39s
	Train Loss: 1.988 | Train PPL:   7.297
	 Val. Loss: 3.447 |  Val. PPL:  31.393
Epoch: 09 | Time: 1m 39s
	Train Loss: 1.812 | Train PPL:   6.121
	 Val. Loss: 3.404 |  Val. PPL:  30.094
Epoch: 10 | Time: 1m 39s
	Train Loss: 1.649 | Train PPL